## Data collection from [NOMADS](https://github.com/microsoft/AIforEarthDataSets#noaa-high-resolution-rapid-refresh-hrrr)


In [1]:
from NOMADS_extraction import NOMADS
from datetime import date, timedelta
import xarray as xr
import matplotlib.pyplot as plt

import os
import numpy as np

In [2]:
# Numpy
# Define the start and end dates
start_date = date(2023, 8, 19)
end_date = date(2023, 8, 30)

# Define the base folder path
base_folder_path = './Data/'

# Create directories for each date if they don't exist
current_date = start_date
while current_date <= end_date:
    date_folder = os.path.join(base_folder_path, current_date.strftime('%Y-%m-%d'))
    os.makedirs(date_folder, exist_ok=True)
    current_date += timedelta(days=1)

# Process data for each date
for day in (start_date + timedelta(n) for n in range((end_date - start_date).days + 1)):
    for cycle in range(24):
        cycle_folder = os.path.join(base_folder_path, day.strftime('%Y-%m-%d'), str(cycle))
        os.makedirs(cycle_folder, exist_ok=True)

        for forecast_hour in range(24):
            print(f"{day}-{cycle}-{forecast_hour} is in progress..")
            try:
                url, r, idx = NOMADS.get_NOMADS_url_idx(day, cycle, forecast_hour)

                # Download and process data for each parameter
                for parameter in ["TMP:2 m", "DSWRF", "RH:2 m", "WIND", "TCDC"]:
                    try:
                        data_file = NOMADS.download_data(idx, parameter, url)
                        xarr = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs={'indexpath': ''})
                        
                        # Extract the variable of interest as a NumPy array
                        variable_name = list(xarr.data_vars)[0]
                        numpy_data = xarr[variable_name].values

                        # Save the NumPy array to a file
                        save_path = os.path.join(cycle_folder, f'{day}_{cycle}_{forecast_hour}_{variable_name}.npy')
                        np.save(save_path, numpy_data)
                    except IndexError:
                        print(f"Data not found for {parameter} on {day}, cycle {cycle}, forecast hour {forecast_hour}")

            except Exception as e:
                print(f"An error occurred on {day}, cycle {cycle}, forecast hour {forecast_hour}: {e}")

    print(f"Completed processing for {day}")


2023-08-19-0-0 is in progress..
2023-08-19-0-1 is in progress..
2023-08-19-0-2 is in progress..
2023-08-19-0-3 is in progress..
2023-08-19-0-4 is in progress..
2023-08-19-0-5 is in progress..
2023-08-19-0-6 is in progress..
2023-08-19-0-7 is in progress..
2023-08-19-0-8 is in progress..
2023-08-19-0-9 is in progress..
2023-08-19-0-10 is in progress..
2023-08-19-0-11 is in progress..
2023-08-19-0-12 is in progress..
2023-08-19-0-13 is in progress..
2023-08-19-0-14 is in progress..
2023-08-19-0-15 is in progress..
2023-08-19-0-16 is in progress..
2023-08-19-0-17 is in progress..
2023-08-19-0-18 is in progress..
2023-08-19-0-19 is in progress..
2023-08-19-0-20 is in progress..
2023-08-19-0-21 is in progress..
2023-08-19-0-22 is in progress..
2023-08-19-0-23 is in progress..
2023-08-19-1-0 is in progress..
2023-08-19-1-1 is in progress..
2023-08-19-1-2 is in progress..
2023-08-19-1-3 is in progress..
2023-08-19-1-4 is in progress..
2023-08-19-1-5 is in progress..
2023-08-19-1-6 is in progr

### Extracting based on the 4 defined locations

In [ ]:
# Define the date range and parameters
start_date = date(2023, 6, 1)
end_date = date(2023, 8, 30)
parameters = ["dswrf", "r2", "si10", "t2m", "tcc"]

# Base folder path
base_folder_path = './Data/'

# Define locations and their coordinates
locations = {
    "Texas": (236, 937),
    "Iowa": (662, 1006),
    "Nevada": (490, 372),
    "Seattle": (994, 268)
}

# Iterate over each forecast hour
for forecast_hour in range(7, 9):  # From 0 to 18
    # Collect data in a list for each forecast hour and all locations
    data_lists = {location: [] for location in locations}

    for day in (start_date + timedelta(n) for n in range((end_date - start_date).days + 1)):
        for cycle in range(24):
            for location, (latitude, longitude) in locations.items():
                row = {'Date': day, 'Cycle': cycle}
                cycle_folder = os.path.join(base_folder_path, day.strftime('%Y-%m-%d'), str(cycle))

                for parameter in parameters:
                    csv_path = os.path.join(cycle_folder, f'{day}_{cycle}_{forecast_hour}_{parameter}.npy')
                    print(csv_path)
                    if os.path.exists(csv_path) and os.path.getsize(csv_path) > 0:
                        try:
                            data = np.load(csv_path)
                            row[parameter] = data[latitude, longitude]
                        except ValueError as e:
                            print(f"Error loading file {csv_path}: {e}")
                            # Handle the error - e.g., set a default value or skip
                    else:
                        print(f"File {csv_path} not found or is empty")
                        # Handle the missing or empty file case

                data_lists[location].append(row)

    # Convert the lists of dictionaries to DataFrames for all locations
    dfs = {location: pd.DataFrame(data_list) for location, data_list in data_lists.items()}

    # Save the DataFrames to CSV files for all locations
    for location, df in dfs.items():
        output_file_path = f'./Data/{location}/weather_data_forecast_{forecast_hour}.csv'
        df.to_csv(output_file_path, index=False)
        print(f"DataFrame for forecast hour {forecast_hour} in {location} saved to {output_file_path}")


### Caluclate WBGT

In [ ]:
# Define the base folder path
base_folder_path = './Data/NOAA/2023_Summer/Texas/'

# Create a dictionary to store DataFrames
dfs = {}

for forecast_hour in range(19):
    file_path = os.path.join(base_folder_path, f'weather_data_forecast_{forecast_hour}.csv')
    
    # Use the f-string to dynamically create DataFrame keys (e.g., 'df_0', 'df_1', etc.)
    df_key = f'df_{forecast_hour}'
    
    # Read the CSV file and store it in the dictionary
    dfs[df_key] = pd.read_csv(file_path)

# Rearrange columns
for forecast_hour in range(19):
    # Use the f-string to dynamically create DataFrame keys (e.g., 'df_0', 'df_1', etc.)
    df_key = f'df_{forecast_hour}'
    
    #Forecast column
    dfs[df_key]['forecast'] = forecast_hour

    #Hour column
    dfs[df_key]['hour'] = dfs[df_key]['forecast'] + dfs[df_key]['Cycle']
    # dfs[df_key]['hour'] = dfs[df_key]['Cycle']

    #Datetime column
    dfs[df_key]['Date'] = pd.to_datetime(dfs[df_key]['Date'])
    pd.to_timedelta(dfs[df_key]['hour'], unit='h')
    dfs[df_key]['datetime'] = dfs[df_key]['Date'] + pd.to_timedelta(dfs[df_key]['hour'], unit='h')
    
    #Drop the columns
    dfs[df_key] = dfs[df_key].drop(['Date', 'Cycle', 'hour'], axis=1)

    #Rename the columns
    dfs[df_key].rename(columns={'dswrf': 'GHI', 'r2': 'rh', 'si10': 'va', 't2m': 'ta', 'tcc': 'cloud'}, inplace=True)

    #Change the order
    new_column_order = ['datetime', 'ta', 'GHI', 'rh', 'va', 'cloud', 'forecast']
    dfs[df_key] = dfs[df_key][new_column_order]

# Combinging df based on datetime
df = pd.concat(dfs, axis=0)
df.reset_index(drop=True, inplace=True)

df = df.sort_values(by=['datetime', 'forecast'])
df = df.reset_index(drop=True)

df['ta'] = df['ta'] - 273.15 # to celcius
df.head()

### Calculating WBGT

In [ ]:
def calculate_wbgt(row):
    """
    Ono, M. & Tonouchi, M. Estimation of wet-bulb globe temperature using generally measured meteorological indices. JAPANESE J. Biometeorol. 50, 147–157 (2014)
    """
    ta = row['ta']
    GHI = row['GHI']
    rh = row['rh']
    va = row['va']    
    GHI =  GHI/1000
    
    wbgt = 0.735 * ta + 0.0374 * rh + 0.00292 * ta * rh + 7.619 * GHI - 4.557 * (GHI ** 2) - 0.0572 * va - 4.064

    return wbgt


# Apply the function to create a new 'WBGT' column
df['WBGT'] = df.apply(calculate_wbgt, axis=1)

### Repeat for other three states